### MTH 610 Model Constrained Optimization 1 (Homework 1)

##### Emily Bogle
##### 10/17/23

##### Problem 1

##### Problem 2
Consider the multistage system obtained by applying Euler's method to the IVP (1-2)


$$
x_{i+1}=x_i+h[a x_i+u_i], \ i=0:N-1 \quad \quad \textrm{(4)}
$$

$$
x_0=\overline{x_0} \quad \quad \quad \quad \quad \quad \quad \quad \quad \quad \quad \quad \quad \quad \quad \textrm{(5)}
$$

with the specified initial value $\overline{x_0}$. The time step $h$ is taken as $h=\frac{1}{N}$, such that $x_N$ is the discrete version of the solution to (1)-(2) at time $t=1$.

The cost functional is also defined as a discrete version to (3),
$$
\tilde{J}(x,u)= \frac{1}{2} x_N^2 +\frac{h}{2} \sum\limits_{i=0}^{N-1}[x_i^2+u_i^2] \quad \quad \quad \quad \textrm{(6)}
$$

- Find the expression of control vector $\mathbf{u}\in \mathbb{R}^N$ the minimizes $\tilde{J}$ subject to the model constraints and with the specified initial condition $\overline{x_0}$.
- Plot the discrete optimal control $u$, state $x$ and the Lagrange multipliers $\lambda$ for each othe following
    - $\overline{x_0}=2, a=1$
    - $\overline{x_0}=2, a=-1$
  Compare with the results obtained in the continuous formulation (Problem 1).

Use $N=100$ for the plots. 
  

We will first find an expression for  $\mathbf{u}\in\mathbb{R}^N$ minimizing $\tilde{J}$ subject to model constraints and with specified initial condition $\overline{x_0}$. We have the following optimization problem:

$$
\min \frac{1}{2}x_N^2+\frac{h}{2}\sum\limits_{i=1}^{N-1}[x_i^2+u_i^2] \quad \quad \quad (*)
$$
$$
\textrm{subject to } x_{i+1}=x_i+h[ax_i+u_i], \ i=0:N-1
$$
$$
x_0=\overline{x_0}
$$

Therefore, the corresponding Lagrangian function is the following, 
$$
\mathcal{L}(x_1,\dots,x_N,u_0,\dots, u_{N-1}, \lambda_1,\dots, \lambda_N)=\frac{1}{2}x_N^2+\sum\limits_{i=0}^{N-1}[x_i^2+u_i^2]+\sum\limits_{i=0}^{N-1}\lambda_{i+1}[x_i+h[ax_i+u_i]-x_{i+1}]
$$
where
$$
\phi(x_N)=\frac{1}{2}(x_N), \quad g_i(x_i,u_i)=x_i^2+u_i^2, \quad f_i(x_i,u_i)=x_i+h[ax_i+u_i]
$$

Now we will work on getting our first order optimality system:
We must have that,
$$
\begin{align}
& \quad \frac{\partial\mathcal{L}}{\partial x_N}=0 \\
&\implies \frac{\partial\phi}{\partial x_N} - \lambda_N=0 \\
&\implies \lambda_N=\frac{\partial \phi}{\partial x_N} \\
&\implies \lambda_N= x_N
\end{align}
$$

For $i=1:N-1$,
$$
\begin{align}
& \quad \frac{\partial\mathcal{L}}{\partial x_i}=0 \\
&\implies \frac{\partial g_i}{\partial x_i}+\lambda_{i+1} \frac{\partial f_i}{\partial x_i} -\lambda_i=0 \\
&\implies \lambda_i=\frac{\partial g_i}{\partial x_i}+\lambda_{i+1} \frac{\partial f_i}{\partial x_i} \\
&\implies \lambda_i=2x_i + (1+ha)\lambda_{i+1}
\end{align}
$$

Now from this, we can also see that 

$$
\lambda_{i+1}=\frac{\lambda_i-2x_i}{1+ha} \quad \textrm{(**)}
$$

And for $i=0:N-1$,
$$
\begin{align}
& \quad \frac{\partial\mathcal{L}}{\partial u_i} =0 \\
&\implies \frac{\partial g_i}{\partial u_i} + \lambda_{i+1}\frac{\partial f_i}{\partial u_i} =0\\
&\implies 2 u_i + \lambda_{i=1}h=0 \\
&\implies 2u_i=-h\lambda_{i+1} \\
&\implies u_i=\frac{-h}{2}\lambda_{i+1}
\end{align}
$$

Using (**) above we can see that,

$$
u_i=\frac{-h}{2}(\frac{\lambda_i-2x_i}{1+ha})
$$

Therefore given $\overline{x_0}=x_0$, $a$, $N$, and $h=\frac{1}{N}$ our first order optimality system to (*) is given by the following:

$$
\begin{cases}
x_{i+1}=x_i+h[ax_i+u_i], \quad \quad &i=0:N-1\\
\lambda_N=x_N \\
\lambda_i=2x_i+(1+ha)\lambda_{i+1}, \quad &i=1:N-1\\
u_i=\frac{-h}{2}\lambda_{i+1},\quad \quad \quad \quad \quad \quad &i=1:N-1
\end{cases}
$$

Now we will write a code that will do a forward pass and backpropogation to help up find the discrete optimal control $u$, state $x$ and the Lagrange multipliers $\lambda$ for the given cases.

First, given an initial parameter vector $u=[u_i]$,($i=1:N-1$), $x_0$, $N$, and $a$ we must do a forward propogation using the model to get the state vector $x=[x_i]$, ($i=0:N$) given we already know $x_0$.

In [5]:
#Import the needed packages

import numpy as np

In [63]:
#Define inputs and constants
x_0=2
N=100
h=1/N
a=1
u_vect=np.zeros((100,1))

In [64]:
# Given u at all stages (i=0:N-1) this function does the forward propogation using the model to get all state stages (i=0:N)
def forward_model(x_0,N,a,u_vect):
    x_vect = np.array([[x_0]])
    x = x_0
    for i in range(N):
        u = u_vect[i, 0]
        x = x+h*(a*x+u)
        x_vect = np.append(x_vect, np.array([[x]]), axis=0)
    return x_vect

In [65]:
forward_model(x_0,N,a,u_vect)

array([[2.        ],
       [2.02      ],
       [2.0402    ],
       [2.060602  ],
       [2.08120802],
       [2.1020201 ],
       [2.1230403 ],
       [2.1442707 ],
       [2.16571341],
       [2.18737055],
       [2.20924425],
       [2.23133669],
       [2.25365006],
       [2.27618656],
       [2.29894843],
       [2.32193791],
       [2.34515729],
       [2.36860886],
       [2.39229495],
       [2.4162179 ],
       [2.44038008],
       [2.46478388],
       [2.48943172],
       [2.51432604],
       [2.5394693 ],
       [2.56486399],
       [2.59051263],
       [2.61641776],
       [2.64258193],
       [2.66900775],
       [2.69569783],
       [2.72265481],
       [2.74988136],
       [2.77738017],
       [2.80515397],
       [2.83320551],
       [2.86153757],
       [2.89015294],
       [2.91905447],
       [2.94824502],
       [2.97772747],
       [3.00750474],
       [3.03757979],
       [3.06795559],
       [3.09863514],
       [3.12962149],
       [3.16091771],
       [3.192

Now we will work on doing the backpropogation to get all our Lagrange Multiplier $\lambda_i$ values for $(i=1:n)$.

We start by initialize $\lambda_N=\nabla \phi(x_N)=x_N$ 

Then for $i=N-1:1$ we will define $\lambda_i$ in terms of $\lambda_{i+1}$.

Once we have our $\lambda$ vector $\lambda=[\lambda_i]$, $(i=1:N)$ we can use it to find our reduced function gradient $\nabla_{u_i}\mathcal{L}$ which will used in the optimization of the reduced cost function depending only on the parameter vector $u$. 

In [71]:
x_vect = forward_model(x_0,N,a,u_vect)
lambda_N = x_vect[N, 0]

def back_propogation(lambda_N, x_vect, h, a):
    lambda_vect = np.array([[lambda_N]])
    lambdaa = lambda_N
    for i in range(N-1,0,-1):
        x = x_vect[i, 0]
        lambdaa = 2*x + (1+h*a)*lambdaa
        lambda_vect = np.append(lambda_vect, np.array([[lambdaa]]), axis=0)
    lambda_vect=np.flip(lambda_vect)
    return lambda_vect

In [72]:
back_propogation(lambda_N, x_vect, h, a)

array([[1255.00100118],
       [1238.57524869],
       [1222.27212742],
       [1206.09002319],
       [1190.02733381],
       [1174.08246892],
       [1158.25384982],
       [1142.53990932],
       [1126.93909158],
       [1111.44985197],
       [1096.0706569 ],
       [1080.79998367],
       [1065.63632035],
       [1050.57816557],
       [1035.62402844],
       [1020.77242833],
       [1006.0218948 ],
       [ 991.3709674 ],
       [ 976.81819555],
       [ 962.36213836],
       [ 948.00136455],
       [ 933.73445227],
       [ 919.55998894],
       [ 905.47657116],
       [ 891.48280452],
       [ 877.5773035 ],
       [ 863.75869133],
       [ 850.02559982],
       [ 836.37666926],
       [ 822.81054827],
       [ 809.32589367],
       [ 795.92137035],
       [ 782.59565113],
       [ 769.34741662],
       [ 756.17535512],
       [ 743.07816247],
       [ 730.05454192],
       [ 717.10320399],
       [ 704.22286639],
       [ 691.41225381],
       [ 678.6700979 ],
       [ 665.995

In [ ]:
lambda_vect = back_propogation(lambda_N, x_vect, h, a)